In [1]:
import pandas as pd
import numpy as np
import h5py
import math

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, regularizers, Flatten
from keras.callbacks import CSVLogger

from sklearn import ensemble, preprocessing, multiclass
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, train_test_split

from collections import Counter

Using TensorFlow backend.


In [2]:
# Transformation

def sqrt(col):
    return list(map(np.sqrt, col));

def freq(col):
    col = np.floor(col)
    counter = Counter(col)
    return [counter.get(elem) for elem in col]


In [76]:
# Globals

# Datasets
#dids = np.load("datasets/indexes.npy")
dids = np.load("indexes_2.npy")

# RF model parameters
seed = 67
transformations = [sqrt, freq]
transformations_name = ["sqrt", "freq"]
trans2target1 = {}
trans2target2 = {}
trans2target3 = {}

# Comrpessed Dataset paramters
qsa_representation = []
num_bin = 10
too_big = 10000

# Neural Nets Parameters and Variables
#MLP_LFE_Nets = {}
inp_shape = (2,num_bin)
dropout = 0.2
norm = (0, 10)
pred_threshold = 0.5
train_set_max = 80000
test_set_max = 80000


In [4]:
#def binarize_dataset():

def load_dataset(id):
    X = np.load("datasets/binary_numeric/" + str(id) + "-data.npy")
    y = np.load("datasets/binary_numeric/" + str(id) + "-target.npy")
    categorical = np.load("datasets/binary_numeric/" + str(id) + "-categorical.npy")
    return X,y,categorical

    
def evaluate_model(X, y, categorical):
    imp = Imputer(missing_values="NaN")
    X = imp.fit_transform(X)
    enc = preprocessing.OneHotEncoder(categorical_features=categorical)
    X = enc.fit_transform(X)
    clf = ensemble.RandomForestClassifier(random_state=seed)
    #clf_ovsr = multiclass.OneVsRestClassifier(clf, n_jobs=-1)
    
    return cross_val_score(clf, X, y,cv=10)
    
def is_positive(X,y,categorical,base_score,transformation,feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    X = np.c_[X,transformed_feature]
    categorical = np.append(categorical,False)
    new_score = evaluate_model(X,y,categorical).mean()
    
    return 1 if(base_score <= (new_score - 0.01)) else 0

def is_positive_2(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(base_score <= (new_score - 0.005)) else 0

def is_positive_3(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(new_score > base_score*1.01) else 0
    
    

In [5]:
# Build the target for the compressed feature
bad_datasets = []

def build_target_for_compressed(dids):

    for transf in transformations:
        trans2target1[transf] = []
        trans2target2[transf] = []
        trans2target3[transf] = []

    
    for did in dids:
        print("Start dataset number", did)
        
        try:

            X, y, categorical = load_dataset(did)       

            new_indexes = []

            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            base_score = evaluate_model(X, y, categorical).mean()

            # Find the indexes of numeric attributes
            numerical_indexes = np.where(np.invert(categorical))[0]
            sample_numerical_indexes = np.random.choice(numerical_indexes, min(numerical_indexes.shape[0],10), replace=False)

            for i,transf in enumerate(transformations):
                for feature in sample_numerical_indexes:

                    print("\tEvaluating feature " + str(feature))

                    mlp_target_1 = is_positive(X,y,categorical,base_score,transf, feature)
                    mlp_target_2 = is_positive_2(X,y,categorical,base_score,transf, feature)
                    mlp_target_3 = is_positive_3(X,y,categorical,base_score,transf, feature)

                    print("\t\t" + str(mlp_target_1), str(mlp_target_2), str(mlp_target_3))

                    trans2target1[transf].append((did,feature,mlp_target_1))
                    trans2target2[transf].append((did,feature,mlp_target_2))
                    trans2target3[transf].append((did,feature,mlp_target_3))

        except:
                print("The evaluation of dataset " + str(did) + " failed")
                bad_datasets.append(did)
                continue

In [6]:
# Save the result
def save_target_for_compressed(path):

    for transf, name in zip(transformations, transformations_name):
        np.save(path + name + "1", trans2target1[transf])
        np.save(path + name + "2", trans2target2[transf])
        np.save(path + name + "3", trans2target3[transf])


In [7]:
def normalize_Rx(matrix):
    
    Rxc = np.zeros(shape=matrix.shape)
    
    for i,row in enumerate(matrix):
        max_c = np.amax(row)
        min_c = np.amin(row)
        bin_width = (max_c-min_c)/(norm[1]-norm[0])
        Rxc[i] = np.apply_along_axis(lambda x : np.floor((x-min_c)/(bin_width)+norm[0]), 0, row)
    
    return Rxc

def to_quantile_sketch_array(did, col, targets, bins, t_class, index):
    max_c = np.nanmax(col)
    min_c = np.nanmin(col)
    bin_width = (max_c-min_c)/num_bin
    Rx = np.zeros(shape=(2,num_bin))
    
    if(bin_width == 0):
        return
    
    for val,y in zip(col,targets):
        if not np.isnan(val):
            bin_value = int(np.floor((val-min_c)/bin_width))
            bin_value = np.clip(bin_value, 0, num_bin-1)
            my_class = 0 if t_class == y else 1
            Rx[my_class][bin_value] = Rx[my_class][bin_value] + 1
            
    Rx = normalize_Rx(Rx)

    qsa_representation.append(np.insert(Rx.flatten(), 0, [did,index]))

In [8]:
# Build the compressed dataset
def build_compressed_dataset(dids):

    for did in dids:
        print("Start dataset number", did)

        try:
            X, y, categorical = load_dataset(did)
        
            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            numerical_indexes = np.where(np.invert(categorical))[0]

            classes = set(y)

            for t_class in classes:
                for index in numerical_indexes:
                    to_quantile_sketch_array(did,X[:,index], y, num_bin, t_class, index)
                
        except:
            print("Error with dataset " + str(did))
            continue;
        
    

In [9]:
# Save the compressed datasets
def save_compressed_dataset(path):
    np.save(path + "compressed.npy", qsa_representation)

In [10]:
# CREATING THE NEURAL NETS

def initialize_MLPs():
    
    for transf in transformations_name:
        model = Sequential()

        model.add(Dense(64, input_shape=inp_shape, W_regularizer=regularizers.l2(0.01)))
        model.add(Activation('softmax'))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(output_dim=1))
        model.add(Activation('relu'))

        # For a binary classification problem
        model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        MLP_LFE_Nets[transf] = model
    

In [41]:

def load_compressed_ds():
    data = pd.DataFrame(np.load("datasets/compressed/compressed.npy"))
    data.columns = [str(i) for i in range (num_bin*2+2)]
    return data


def assemble_training_set(compressed, transformation_targets):
    targetDf = pd.DataFrame(transformation_targets)
    targetDf.columns = ["0","1", "2"]
    merged = pd.merge(compressed, targetDf, how='left', on=["0", "1"])
    class_1 = merged.ix[:,2:num_bin + 2].values
    class_2 = merged.ix[:,num_bin + 2:-1].values
    target = np.array(merged.ix[:,-1].values)
    meta_inf = np.array(merged.ix[:,:2].values)
    meta_target = np.c_[target, meta_inf]
    X = []
    
    for c1, c2 in zip(class_1, class_2):
        X.append([c1,c2])    

    return np.array(X), np.array(meta_target)


def split_training_test():
    compressed_ds = load_compressed_ds()

    for transf, name in zip(transformations, transformations_name):
        transformation_targets = np.load("datasets/compressed/" + name + "2.npy")
        X,y = assemble_training_set(compressed_ds, transformation_targets)
        X_s_tr, X_s_test, y_s_tr, y_s_test = train_test_split(X, y, test_size=0.3)
        
        # Dropping the meta-info from training set
        y_s_tr = y_s_tr[:,:1]
        
        np.save("datasets/training/" + name + "-data_split",X_s_tr)
        np.save("datasets/training/" + name + "-target_split",y_s_tr)
        np.save("datasets/test/" + name + "-data_split",X_s_test)
        np.save("datasets/test/" + name + "-target_split",y_s_test)
 
        
def load_training_set(transf):
    
    X = np.load("datasets/training/" + transf + "-data_split.npy")
    y = np.load("datasets/training/" + transf + "-target_split.npy")
    
    return X,y


In [12]:
def balance_dataset(X, y, pos_perc = 0.5):
    X = np.array(X)
    y = np.array(y)

    cnt = Counter(y)

    neg_num = cnt[0]
    pos_num = cnt[1]

    neg_index = (y == 0)
    pos_index = (y == 1)

    X_pos = X[pos_index]
    X_neg = X[neg_index]
    y_pos = y[pos_index]
    y_neg = y[neg_index]

    if pos_perc >= 0.5:
        if pos_num <= neg_num:
            sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
            neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
            X_neg = X_neg[neg_sampled_indexes]
            y_neg = y_neg[neg_sampled_indexes]

        else:
            sample_neg_ideal_size = int(X_pos.shape[0] * (1/pos_perc - 1))

            if(sample_neg_ideal_size > X_neg.shape[0]):
                sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
            else:
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_ideal_size, replace=False) 
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
    else:
        if pos_num <= neg_num:
            sample_pos_ideal_size = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))

            if(sample_pos_ideal_size > X_pos.shape[0]):
                sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
            else:
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_ideal_size, replace=False) 
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
        else:
            sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
            pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
            X_pos = X_pos[pos_sampled_indexes]
            y_pos = y_pos[pos_sampled_indexes]

    X = np.concatenate((X_pos, X_neg), axis=0)
    y = np.concatenate((y_pos, y_neg) , axis=0)      

    shuffle_index = np.random.choice(X.shape[0], X.shape[0], replace=False)

    X = X[shuffle_index]
    y = y[shuffle_index]

    return X,y

In [80]:
# Training the nets

def train_MLPs():

    for transf, name in zip(transformations, transformations_name):
        print("Start training " + name)
        
        X, y = load_training_set(name)
        
        #if(X.shape[0] > train_set_max):
        #        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        #        X = X[new_indexes]
        #        y = y[new_indexes]
        
        X, y = balance_dataset(X, y.reshape(y.shape[0]), pos_perc=0.65)
        print("dataset_balanced")
        
        y = y.reshape(y.shape[0],1)

        MLP_LFE_Nets[name].summary()
        print ("Inputs: {}".format(MLP_LFE_Nets[name].input_shape))
        print ("Outputs: {}".format(MLP_LFE_Nets[name].output_shape))
        print ("Actual input: {}".format(X.shape))
        print ("Actual output: {}".format(y.shape))
        
        csv_logger = CSVLogger("datasets/MLPs/log/log-"+ name +".csv", append=True, separator=';')
        MLP_LFE_Nets[name].fit(X, y, epochs=100, verbose=1, validation_split=0.2 ,callbacks=[csv_logger]) 


In [14]:

def save_MLPs():
    for transf in transformations_name:
        MLP_LFE_Nets[transf].save_weights("datasets/MLPs/" + transf + "-weights")
        model_json = MLP_LFE_Nets[transf].to_json()
        with open("datasets/MLPs/" + transf + "-net_model", "w") as f:
            f.write(model_json)

def load_MLPs():
    
    for name in transformations_name:
        json_file = open('datasets/MLPs/' + name + "-net_model", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        
        # load weights into new model
        loaded_model.load_weights("datasets/MLPs/" + name + "-weights")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                             loss='binary_crossentropy',
                             metrics=['accuracy'])
        
        MLP_LFE_Nets[name] = loaded_model


In [15]:
def load_test_set():
    
    X = np.load("datasets/test/" + transformations_name[0] + "-data_split.npy")
    y_meta = np.load("datasets/test/" + transformations_name[0] + "-target_split.npy")
    t = np.full((y_meta.shape[0],1), 0)
    y_meta = np.concatenate((y_meta, t), axis=1)
    
    for i,name in enumerate(transformations_name[1:]):
        X = np.concatenate((X, np.load("datasets/test/" + name + "-data_split.npy")), axis=0)
        y_meta_tmp = np.load("datasets/test/" + name + "-target_split.npy")
        t = np.full((y_meta_tmp.shape[0],1),i+1)
        y_meta_tmp = np.concatenate((y_meta_tmp, t), axis=1)
        y_meta = np.concatenate((y_meta, y_meta_tmp), axis=0)
    
    return X,y_meta

In [16]:
def evaluate_transformation_classifier():
    
    # Number of prediction on features
    num_of_prediction = {}
    # Number of correct prediction on features
    num_of_correct_prediction = {}
    # Number of dataset which received a prediction
    good_predicted_dids = set()
    num_of_predicted_dataset = 0
    
    pred_mat = []
    
    X, y_meta = load_test_set()
    
    if(X.shape[0] > test_set_max):
        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        X = X[new_indexes]
        y_meta = y_meta[new_indexes]
    
    for transf in transformations_name:
        pred_mat.append(MLP_LFE_Nets[transf].predict(X))
        num_of_prediction[transf] = 0
        num_of_correct_prediction[transf] = 0
        
    pred_mat = np.array(pred_mat).transpose()
    
    for predictions,did,feature in zip(pred_mat[0],y_meta[:,1], y_meta[:,2]):
        pmax = np.amax(predictions)
        print(predictions)
        
        if pmax > pred_threshold:
            index = np.where(predictions==pmax)[0][0]
            print(index)
            num_of_prediction[transformations_name[index]] += 1
            
            # Select the target for the transformation and the dataset
            positive_example_found =  np.where((y_meta[:,0] == 1)       &  \
                                               (y_meta[:,1] == did)     &  \
                                               (y_meta[:,2] == feature) &  \
                                               (y_meta[:,3] == index))     \
                                                                       [0].shape[0] > 0
            
            if(positive_example_found):
                good_predicted_dids.add(did) 
                num_of_correct_prediction[transformations_name[index]] += 1
    
    
    
    for transf in transformations_name:
        
        if(num_of_prediction[transf] == 0):
            print("No predictions have been made")
            continue
    
        print("Evalutation of the transformation classifier: " + transf)
        print("\tNumber of prediction:", num_of_prediction[transf])
        print("\tNumber of Correct prediciton:", num_of_correct_prediction[transf])
        print("Accuracy:", num_of_correct_prediction[transf]/num_of_prediction[transf])
        
    print("\n")    
    print("Number of datasets who received a good prediction:", len(good_predicted_dids))

# Build and Preprocess the Dataset

In [ ]:

build_target_for_compressed(dids)
save_target_for_compressed("datasets/compressed/")
    

In [ ]:

build_compressed_dataset(dids)
save_compressed_dataset("datasets/compressed/")


# Train Test Split

In [72]:

split_training_test()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':



# Train the MLPs


In [81]:

initialize_MLPs()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_shape=(2, 10), kernel_regularizer=<keras.reg...)`
  
C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  if sys.path[0] == '':


In [82]:

train_MLPs()


Start training sqrt
dataset_balanced
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 2, 64)             704       
_________________________________________________________________
activation_49 (Activation)   (None, 2, 64)             0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 2, 64)             0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 129       
_________________________________________________________________
activation_50 (Activation)   (None, 1)                 0         
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________

163/163 [==============================] - 0s - loss: 0.7119 - acc: 0.6074 - val_loss: 0.6539 - val_acc: 0.7073
Epoch 57/100
163/163 [==============================] - 0s - loss: 0.6753 - acc: 0.6748 - val_loss: 0.6497 - val_acc: 0.7073
Epoch 58/100
163/163 [==============================] - 0s - loss: 0.6874 - acc: 0.6319 - val_loss: 0.6460 - val_acc: 0.7073
Epoch 59/100
163/163 [==============================] - 0s - loss: 0.6688 - acc: 0.6380 - val_loss: 0.6439 - val_acc: 0.7073
Epoch 60/100
163/163 [==============================] - 0s - loss: 0.6996 - acc: 0.6258 - val_loss: 0.6447 - val_acc: 0.7073
Epoch 61/100
163/163 [==============================] - 0s - loss: 0.6985 - acc: 0.6074 - val_loss: 0.6436 - val_acc: 0.7073
Epoch 62/100
163/163 [==============================] - 0s - loss: 0.6769 - acc: 0.6748 - val_loss: 0.6397 - val_acc: 0.7073
Epoch 63/100
163/163 [==============================] - 0s - loss: 0.6752 - acc: 0.6380 - val_loss: 0.6413 - val_acc: 0.7073
Epoch 64/100


252/252 [==============================] - 0s - loss: 1.4104 - acc: 0.3373 - val_loss: 1.2921 - val_acc: 0.4062
Epoch 12/100
252/252 [==============================] - 0s - loss: 1.3314 - acc: 0.3651 - val_loss: 1.2219 - val_acc: 0.4062
Epoch 13/100
252/252 [==============================] - 0s - loss: 1.2621 - acc: 0.3690 - val_loss: 1.1596 - val_acc: 0.4062
Epoch 14/100
252/252 [==============================] - 0s - loss: 1.1587 - acc: 0.3968 - val_loss: 1.1040 - val_acc: 0.4219
Epoch 15/100
252/252 [==============================] - 0s - loss: 1.1121 - acc: 0.3810 - val_loss: 1.0543 - val_acc: 0.4531
Epoch 16/100
252/252 [==============================] - 0s - loss: 1.0271 - acc: 0.4048 - val_loss: 1.0096 - val_acc: 0.4531
Epoch 17/100
252/252 [==============================] - 0s - loss: 0.9950 - acc: 0.4167 - val_loss: 0.9704 - val_acc: 0.4062
Epoch 18/100
252/252 [==============================] - 0s - loss: 0.9666 - acc: 0.3532 - val_loss: 0.9359 - val_acc: 0.3594
Epoch 19/100


252/252 [==============================] - 0s - loss: 0.5326 - acc: 0.7579 - val_loss: 0.6037 - val_acc: 0.7500
Epoch 77/100
252/252 [==============================] - ETA: 0s - loss: 0.4198 - acc: 0.843 - 0s - loss: 0.5303 - acc: 0.7579 - val_loss: 0.6069 - val_acc: 0.7500
Epoch 78/100
252/252 [==============================] - 0s - loss: 0.5427 - acc: 0.7659 - val_loss: 0.5958 - val_acc: 0.7656
Epoch 79/100
252/252 [==============================] - 0s - loss: 0.5283 - acc: 0.7659 - val_loss: 0.5978 - val_acc: 0.7500
Epoch 80/100
252/252 [==============================] - 0s - loss: 0.5388 - acc: 0.7738 - val_loss: 0.6005 - val_acc: 0.7500
Epoch 81/100
252/252 [==============================] - 0s - loss: 0.5409 - acc: 0.7619 - val_loss: 0.5993 - val_acc: 0.7500
Epoch 82/100
252/252 [==============================] - 0s - loss: 0.5460 - acc: 0.7579 - val_loss: 0.5972 - val_acc: 0.7500
Epoch 83/100
252/252 [==============================] - 0s - loss: 0.5351 - acc: 0.7619 - val_loss: 

In [19]:

save_MLPs()


# Load the nets

In [24]:

load_MLPs()



# Test the nets


In [75]:

evaluate_transformation_classifier()


[ 0.          0.27005392]
[ 0.          0.30771509]
[ 0.          0.31157911]
[ 0.          0.22749865]
[ 0.          0.29285595]
[ 0.          0.21779415]
[ 0.          0.29415619]
[ 0.         0.2852127]
[ 0.          0.28336912]
[ 0.         0.3074846]
[ 0.          0.27775246]
[ 0.          0.27030581]
[ 0.         0.3133375]
[ 0.          0.26109865]
[ 0.         0.3048847]
[ 0.          0.32441592]
[ 0.          0.27421123]
[ 0.          0.26156154]
[ 0.          0.26568288]
[ 0.          0.27775246]
[ 0.          0.22110583]
[ 0.          0.27139604]
[ 0.          0.26625657]
[ 0.          0.33692592]
[ 0.          0.32413912]
[ 0.          0.21832104]
[ 0.          0.30388379]
[ 0.          0.27421123]
[ 0.          0.33104998]
[ 0.          0.31936526]
[ 0.          0.27421123]
[ 0.          0.26692301]
[ 0.          0.34187067]
[ 0.          0.32038814]
[ 0.          0.34758741]
[ 0.01960238  0.18645686]
[ 0.          0.26302212]
[ 0.          0.27385095]
[ 0.          0.2825

[ 0.          0.31991869]
[ 0.          0.28607428]
[ 0.          0.27775246]
[ 0.          0.34379435]
[ 0.          0.27775246]
[ 0.          0.33195132]
[ 0.          0.25696206]
[ 0.          0.16200614]
[ 0.          0.27139604]
[ 0.          0.26481253]
[ 0.          0.33133966]
[ 0.          0.27201509]
[ 0.          0.26540744]
[ 0.          0.30111289]
[ 0.          0.27775246]
[ 0.          0.25668731]
[ 0.          0.04442443]
[ 0.          0.32132983]
[ 0.          0.33352995]
[ 0.          0.27775246]
[ 0.          0.28209692]
[ 0.          0.27971783]
[ 0.          0.28538579]
[ 0.          0.33755952]
[ 0.          0.31095433]
[ 0.          0.29522455]
[ 0.          0.28582257]
[ 0.          0.31113827]
[ 0.          0.32530874]
[ 0.          0.27829063]
[ 0.        0.344217]
[ 0.         0.2770822]
[ 0.          0.24799849]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.33855653]
[ 0.          0.25919226]
[ 0.          0.31707293]
[ 0.         0.317

[ 0.         0.2894077]
[ 0.          0.27536708]
[ 0.          0.24481103]
[ 0.          0.25866002]
[ 0.         0.2765435]
[ 0.          0.28607428]
[ 0.          0.31420439]
[ 0.          0.29348332]
[ 0.          0.27787724]
[ 0.          0.28607428]
[ 0.          0.26574779]
[ 0.          0.28582257]
[ 0.          0.21747623]
[ 0.          0.32176083]
[ 0.          0.26966336]
[ 0.          0.27775246]
[ 0.          0.30835816]
[ 0.          0.27419561]
[ 0.          0.28958133]
[ 0.          0.28779972]
[ 0.          0.19623435]
[ 0.          0.26945919]
[ 0.          0.29271984]
[ 0.          0.23742397]
[ 0.         0.2987617]
[ 0.          0.24891666]
[ 0.          0.28136376]
[ 0.          0.25046951]
[ 0.          0.32572323]
[ 0.          0.27774066]
[ 0.          0.27775246]
[ 0.          0.22643828]
[ 0.         0.2948693]
[ 0.          0.28209692]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.32642072]
[ 0.          0.33524662]
[ 0.          0.3133

[ 0.          0.28574693]
[ 0.         0.2747764]
[ 0.          0.27355248]
[ 0.          0.30875099]
[ 0.          0.32485598]
[ 0.          0.19579196]
[ 0.          0.20679425]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.33025897]
[ 0.         0.1829997]
[ 0.          0.29061493]
[ 0.          0.29069012]
[ 0.          0.27775246]
[ 0.         0.2080642]
[ 0.          0.26904032]
[ 0.          0.26584786]
[ 0.          0.24491018]
[ 0.         0.3449921]
[ 0.          0.27421123]
[ 0.          0.27775246]
[ 0.          0.33699459]
[ 0.          0.29291657]
[ 0.          0.26793128]
[ 0.          0.27421123]
[ 0.          0.27775246]
[ 0.          0.32513982]
[ 0.          0.27139604]
[ 0.          0.27040195]
[ 0.          0.27775246]
[ 0.          0.27665281]
[ 0.          0.32290703]
[ 0.          0.27775246]
[ 0.          0.29032791]
[ 0.          0.28119326]
[ 0.          0.29415619]
[ 0.          0.34161025]
[ 0.          0.3395

[ 0.          0.32586747]
[ 0.06557028  0.04871485]
[ 0.          0.31314826]
[ 0.          0.27775246]
[ 0.          0.29661843]
[ 0.          0.27421123]
[ 0.          0.34163159]
[ 0.          0.19708373]
[ 0.          0.31209385]
[ 0.          0.27421123]
[ 0.          0.30416036]
[ 0.          0.29547888]
[ 0.          0.29552531]
[ 0.         0.3174082]
[ 0.          0.26795626]
[ 0.          0.27883983]
[ 0.          0.23724209]
[ 0.          0.29061493]
[ 0.          0.27159923]
[ 0.          0.30629182]
[ 0.          0.31825465]
[ 0.          0.27775246]
[ 0.          0.22516173]
[ 0.          0.31444329]
[ 0.00931633  0.18974587]
[ 0.          0.28486237]
[ 0.          0.21781425]
[ 0.          0.28253305]
[ 0.          0.30661398]
[ 0.          0.22515397]
[ 0.          0.27562112]
[ 0.          0.24955301]
[ 0.          0.33048868]
[ 0.          0.32684875]
[ 0.          0.27775246]
[ 0.          0.28607428]
[ 0.          0.31112379]
[ 0.          0.27775246]
[ 0.          

[ 0.          0.29236889]
[ 0.          0.27421123]
[ 0.          0.15142925]
[ 0.          0.27296776]
[ 0.          0.28779972]
[ 0.          0.30657369]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.21979962]
[ 0.          0.32962191]
[ 0.          0.30005121]
[ 0.          0.28134304]
[ 0.          0.27775246]
[ 0.          0.26246047]
[ 0.          0.27269104]
[ 0.          0.26640278]
[ 0.          0.21837103]
[ 0.          0.32623088]
[ 0.          0.27775246]
[ 0.          0.20651622]
[ 0.          0.23739699]
[ 0.         0.3430106]
[ 0.          0.27810419]
[ 0.          0.28390822]
[ 0.          0.29640639]
[ 0.          0.27775246]
[ 0.          0.19366607]
[ 0.          0.30225027]
[ 0.          0.31533062]
[ 0.          0.31682193]
[ 0.          0.27775246]
[ 0.          0.32023174]
[ 0.          0.29915601]
[ 0.          0.29945368]
[ 0.          0.28427216]
[ 0.          0.28607428]
[ 0.          0.33473086]
[ 0.          0.20943472]
[ 0.          

[ 0.          0.32265341]
[ 0.          0.27775246]
[ 0.          0.28444728]
[ 0.          0.16946334]
[ 0.          0.28607428]
[ 0.          0.31416976]
[ 0.          0.23742105]
[ 0.          0.31255323]
[ 0.          0.28481525]
[ 0.          0.28607428]
[ 0.          0.32742989]
[ 0.          0.33350217]
[ 0.          0.19254248]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.30974799]
[ 0.          0.28491148]
[ 0.          0.27775246]
[ 0.          0.27421123]
[ 0.          0.23978607]
[ 0.         0.1822053]
[ 0.          0.27667665]
[ 0.          0.27987254]
[ 0.          0.28607428]
[ 0.          0.31256431]
[ 0.          0.19271207]
[ 0.          0.27775246]
[ 0.          0.27775246]
[ 0.          0.28481525]
[ 0.          0.18787304]
[ 0.          0.26263916]
[ 0.          0.28582257]
[ 0.          0.28328484]
[ 0.          0.29168463]
[ 0.          0.33275056]
[ 0.          0.27584454]
[ 0.          0.25072908]
[ 0.          0.31329316]
[ 0.          

In [ ]:
X, y = load_training_set("sqrt")

In [ ]:
X